In [ ]:
### using python 3.10.5
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from scipy.stats import skew
# from scipy.stats import kurtosis
# from scipy.stats import chi2_contingency
from matplotlib import pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate


In [ ]:
### import the data
# there are no headers in the file so they will be imported separately and then appended to the train/test data
# get columns before importing data so we can assign the column headers at import, you can either extract from the txt file or copy it over
# we need to remove the columns containing '| instance weight' according to the instructions and add a column header for the target variable

path_train = r"C:\Users\wausa\Downloads\drive-download-20250217T195304Z-001\census_income_learn.csv"

headers = ['age', 'class of worker', 'detailed industry recode', 'detailed occupation recode', 'education', 'wage per hour', 'enroll in edu inst last wk', 'marital stat',
           'major industry code', 'major occupation code', 'race', 'hispanic origin', 'sex', 'member of a labor union', 'reason for unemployment',
           'full or part time employment stat', 'capital gains', 'capital losses', 'dividends from stocks', 'tax filer stat', 'region of previous residence',
           'state of previous residence', 'detailed household and family stat', 'detailed household summary in household', '| instance weight', 'instance weight',
           'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg', 'live in this house 1 year ago', 'migration prev res in sunbelt',
           'num persons worked for employer', 'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self', 'citizenship',
           'own business or self employed', "fill inc questionnaire for veteran's admin", 'veterans benefits', 'weeks worked in year', 'year']
headers.remove('| instance weight')
headers.append('target') # the final column is the target variable

continuous_data = ['age', 'wage per hour', 'capital gains', 'capital losses', 'dividends from stocks', 'num persons worked for employer', 'weeks worked in year']

df = pd.read_csv(path_train, names=headers)
df.shape

# check the number of records and features

In [ ]:
# look at the continuous data, to determine figures asssoicated to each feature
df.describe()

# number of missing datapoints for each feature 
df.isnull().sum().value_counts() # shown as zero but looking at the data we have a number of missing records marked as '?' or so we 
# will replace the '?' values with NaN. It is also helpful to make the values NaN for encoding later.


,age,detailed industry recode,wage per hour,capital gains,capital losses,dividends from stocks,instance weight,num persons worked for employer,own business or self employed,veterans benefits,weeks worked in year,year,target
count,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000
mean,34.494199,15.352320,-0.134446,-0.060947,-0.104827,-0.060261,1740.380269,0.755334,0.175438,1.514833,23.174897,94.499672,0.062058
std,22.310895,18.067129,0.382961,0.225632,0.304645,0.231729,993.768156,0.807528,0.553694,0.851473,24.411488,0.500001,0.241261
min,0.000000,0.000000,-0.225182,-0.097108,-0.147614,-0.104864,37.870000,0.000000,0.000000,0.000000,0.000000,94.000000,0.000000
25%,15.000000,0.000000,-0.225182,-0.097108,-0.147614,-0.104864,1061.615000,0.000000,0.000000,2.000000,0.000000,94.000000,0.000000
50%,33.000000,0.000000,-0.225182,-0.097108,-0.147614,-0.104864,1618.310000,0.693147,0.000000,2.000000,8.000000,94.000000,0.000000
75%,50.000000,33.000000,-0.225182,-0.097108,-0.147614,-0.104864,2188.610000,1.609438,0.000000,2.000000,52.000000,95.000000,0.000000
max,90.000000,51.000000,3.615560,3.099870,2.879785,3.937674,18656.300000,1.945910,2.000000,2.000000,52.000000,95.000000,1.000000


In [ ]:
df = df.replace('?', np.nan) # we can see now that the empty values are now accounted for

# data is full of whitespaces so they will be removed
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# the target column has been split into two columns (value +50,000 or -50,000), 
# we can avoid this by just converting the value to 1 or 0 in the column. The column is a string so we can just use the string value as the condition

df['target'] = df['target'].apply(lambda x: 0 if x == '- 50000.' else 1)
df['target'].value_counts()

# imbalance in the target classes so we need to consider using some alternative method of evaluating the results aside from the 
# standard confusion matrix, something like an ROC curve

# it may be pertinent to separate the data into numerical and categorical variables to determine the correlations between variables as 
# from the metadata we can see what variables are continues and which are nominal (outlined at the top of the file)

df_continuous = df[continuous_data] # 7 continuous features
df_categorical = df.drop(continuous_data, axis=1) # 35 categorical features
df_categorical = df_categorical.astype('category')


In [ ]:
# there are 35 categorical features, which is high and will get higher if we use one hot encoding for example when we need to process the data
# we can visualise the data
# We have a large amount of zero values, 
# but those zeroes are useful records and indicative of actual values such as when 'wage per hour' is zero, it 
# is due to uneployment

fig, ax = plt.subplots(5, 5)
ax = ax.flatten()         # Convert axes to 1d array of length 9
fig.set_size_inches(15, 25)

for ax, col in zip(ax, df_categorical.columns):
  sns.histplot(df_categorical[col], ax = ax,)
  ax.tick_params(axis='x', labelrotation=45, labelsize=6)

In [ ]:
# there's a lot of categorical variables, and we can use 
# correlation to determine if we can reduce the feature count

# we can view this correlation using Cramers V test, which is used to 
# calcualte the correlation between categorical values, in this 
# example, those which are nominal

def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - (k-1)*(r-1)/(n-1))
    rcorr = r - (r-1)**2/(n-1)
    kcorr = k - (k-1)**2/(n-1)
    return np.sqrt((phi2corr / min( (kcorr-1), (rcorr-1))))

# one example of categorical variable correlation
contingency = pd.crosstab(df_categorical['region of previous residence'], df_categorical['state of previous residence']).values
cramers = cramers_v(contingency)
print(f'Cramer\'s V correlation for \'region of previous residence\' and \'state of previous residence\' is {cramers}')


In [ ]:
# determine the correlation for the categorical features using Cramer's V

df_categorical_first15 = df_categorical.iloc[:, 0:15] # first 15 features, not enough memory to compare them all visually

rows= []
for var1 in df_categorical_first15:
    col = []
    for var2 in df_categorical_first15 :
        contingency = pd.crosstab(df_categorical_first15[var1], df_categorical_first15[var2]).values
        cramers = cramers_v(contingency)
        col.append(round(cramers,2))
    rows.append(col)

cramers_results = np.array(rows)
cramers_df = pd.DataFrame(cramers_results, columns = df_categorical_first15.columns, index =df_categorical_first15.columns)

sns.heatmap(cramers_df)

In [ ]:
# we can drop the highly correlated fields to reduce the number of features
# i.e., detailed industry recode and detailed occupation recode, major industry code and major occupation code, 
# detailed household and family stat and detailed household summary in household
# migration code-change in msa, migration code-change in reg, migration code-move within reg, migration prev res in sunbelt
# fill inc questionnaire for veteran\'s admin and veterans benefits

corr_cols_to_remove = ['detailed occupation recode', 'major occupation code', 'detailed household summary in household', 'migration code-change in msa', 
                     'migration code-change in reg', 'migration code-move within reg', 'migration prev res in sunbelt', 'fill inc questionnaire for veteran\'s admin', 
                     'region of previous residence']

df = df.drop(corr_cols_to_remove, axis=1) # remove columns from original df

df_categorical = df_categorical.drop(corr_cols_to_remove, axis=1)


In [ ]:
# for numerical values we can immediately view the correlation as a heatmap

sns.heatmap(df_continuous.corr(), annot=True, fmt='.2f')

# we can see the numerical values are not necessarily correlated aside from 'number of persons worked for employer'/'weeks worked' 
# which has a fairly strong correlation at 0.75. We will keep all columns as the correlation is not that extreme

In [ ]:
# we may consider scaling the continuous values if the range is high and we can also transform the same data if there's a high skew >0.5 or <-0.5

# we can view the caegorical values
fig, ax = plt.subplots(4, 2)
ax = ax.flatten()         # Convert axes to 1d array of length 9
fig.set_size_inches(15, 25)

for ax, col in zip(ax, df_continuous.columns):
  sns.histplot(df_continuous[col], ax = ax,)
  ax.tick_params(axis='x', labelrotation=45)

In [ ]:
# We can see a lot of zero values here which provides a large amount of skew for this data, however given the nature of the data 
# it doesn't necessarily make sense to remove the data, rather it provides information about people who may be unemployed or have no investments

# for example, most of those not working are either young (<20) or old/retired (> 60) as shown below
df.loc[df['weeks worked in year'] == 0 , 'age'].hist()

In [ ]:
# we can still look at the skew and kurotsis (removed due to length of output) of these continuous variables
# skew above 0.5 or below -0.5 is generally significant and for Kurtosis values above 2 or below -2 

for x in df_continuous.columns:
    print(f'\nThe skew of the variable {x} is: {skew(df[x])}')
    # print(f'The kurtosis of the variable {x} is: {kurtosis(df[x])}')

# we may consider applying a log transformation to reduce the skew.
# we may also want to scale 'wage per hour', 'capital gains', 'capital losses', 'dividends from stocks' as they have large value ranges

scaler = StandardScaler() # scale specific columns -> skew is unchanged

cols_to_transform = ['wage per hour', 'capital gains', 'capital losses', 'dividends from stocks']
for col in cols_to_transform:
    df[col] = scaler.fit_transform(df[[col]])

print(f'\nAfter transforming: {cols_to_transform}')
# now log transform the highly skewed columns 'wage per hour', 'capital gains', 'capital losses', 'dividends from stocks', 'num persons worked for employer'
for col in ['wage per hour', 'capital gains', 'capital losses', 'dividends from stocks', 'num persons worked for employer']:
    df[col] = np.log(df[[col]] + 1) # we need to add the 1 to the values as we cannot log transform the 0 values
    # df[col] = np.sqrt(df[[col]] + 1) # culd sqrt transform instead, again, we need to add the 1 to the values as we cannot sqrt transform the 0 values

# just looking at skew, skews are still large due to the large number of zeroes, but they are reduced
for col in cols_to_transform:
    print(f'\nThe transformed skew of the variable {col} is: {skew(df[col])}')



In [ ]:
# one hot encode data for the categorical values 

ohe_df = pd.get_dummies(df.drop('target', axis=1), dtype=np.uint8)
target_df = df[['target']]

# one hot encoding produces 300+ features which is a lot of features, 
# however we can still use sklearn for this, as a simple model

In [ ]:
# we have a class imbalance for the target variable 
target_df.value_counts()

# so we can choose to over or undersample the target. 
# However, this would mean we would lose a lot of data due to the 
# significance of the imbalance We can adjust the class weights in 
# the loss function instead, that way, we can keep all the data

# additionally, during evaluation, by using metrics that are robust to class 
# imbalance, such as ROC and AUC


In [ ]:
# logistic regression with an L1 penalty, which is the Lasso regrssion, which 
# can shirink the coefficient of 'unimportant' features to 0 

# whilst the continuous features have been transformed, it may be better to 
# transform them to closer represent a normal distributions, bu using a quantile 
# transformation for example, but we have a lot of useful zero data 

# play around with the strength of the L1 penalty

model = LogisticRegression(penalty='l1', solver='liblinear', verbose=2)
cv = cross_validate(model, ohe_df, target_df['target'], scoring='roc_auc', cv=5, return_estimator=True)

In [ ]:
# model evaluated using ROC and AUC with a 5 fold cross validation 
# (again due to the class imbalance) 
cv 

In [ ]:
# presenting the data

log_odds = np.exp(cv['estimator'][0].coef_)[0]
sort_idx = np.argsort(log_odds)

dict(zip([ohe_df.columns[idx] for idx in sort_idx], log_odds[sort_idx]))

In [ ]:
# We can evaluate the model by looking at the the regression coefficients for various variables

# some of the most important featues for those making more than $50k p.a., include:
# 'education_Doctorate degree(PhD EdD)',
# 'education_Prof school degree (MD DDS DVM LLB JD)'
# whilst some features that result in a reduction in the probability of making more than $50k p.a., include:
# 'education_Less than 1st grade'
# 'class of worker_Without pay'


# Because we have used OHE, the figures represent the change in odds 
# associated with the specific feature, rather than resulting in a unit change
# in probability as would be expected from a continuous variable

# things to try if given more time
# reduce overall dimensionality of the data, i.e., using PCA, but without losing too much information
# Scaling/transforming categorical data
# investigate and remove additional features by finding correaltion between continuous and categorical values
# deep learning if willing to invest in more data/time